# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. They are certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
# import pyspark
from pyspark.sql import SparkSession

In [3]:
# Create a SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [4]:
# read in the data hack_data.csv
data = spark.read.csv('hack_data.csv', header=True, inferSchema=True)


In [6]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [8]:
# Show the DataFrame
data.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [9]:
# check columns
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [10]:
# import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
# Create a list of the feature column names
feature_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
                 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed']

In [12]:
# Create a VectorAssembler object
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

In [15]:
# Use the assembler to transform our DataFrame to the two columns: label and features
output = assembler.transform(data)

# Print the resulting schema
output.printSchema()


root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
# standardize the data
from pyspark.ml.feature import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)

In [19]:
# Fit the DataFrame to the scaler
scalerModel = scaler.fit(output)

# Transform the data in features column
scaledData = scalerModel.transform(output)

# Check the resulting column
scaledData.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaledFeatures=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]))]

In [20]:
# Import KMeans
from pyspark.ml.clustering import KMeans

# Create 2 different KMeans models with K=2 and K=3
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)

# Fit the models to the data
model2 = kmeans2.fit(scaledData)
model3 = kmeans3.fit(scaledData)

In [24]:
# Evaluate clustering
from pyspark.ml.evaluation import ClusteringEvaluator

# Make predictions
predictions2 = model2.transform(scaledData)
predictions3 = model3.transform(scaledData)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

# Evaluate the two models and print the scores
silhouette2 = evaluator.evaluate(predictions2)
silhouette3 = evaluator.evaluate(predictions3)

print("Silhouette with squared euclidean distance = " + str(silhouette2))
print("Silhouette with squared euclidean distance = " + str(silhouette3))

Silhouette with squared euclidean distance = 0.6683623593283755
Silhouette with squared euclidean distance = 0.3068084951287429


In [26]:
# count the predictions
predictions2.groupBy('prediction').count().show()
predictions3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   83|
|         0|   84|
+----------+-----+



The last key fact mentioned by the engineer was that the attacks should be evenly numbered between the hackers. This means that the hackers should be evenly distributed between the clusters. Our clustering showed two equal sized clusters with k=2, therefore there were only 2 hackers!